# Day 21: Green Energy

Contribution to the 2024 [#30DayChartChallenge](https://github.com/30DayChartChallenge/Edition2024): Visualizing Germanys (slow) transformation to renewable electricity.

## What is this about?

For many years, German governments have pledged to leave fossil sources behind to generate electricity. The data shows that there have been some successes. In January 2022, I analyzed the data from the German Federal Network Agency (Bundesnetzagentur) and created a series of plots to visualize the transformation to renewable electricity. Read more about the analysis in an [article published at medium.com](https://yotka.medium.com/visualizing-germanys-slow-transformation-to-renewable-electricity-381289bed264).

For the 30DayChartChallenge, I updated the data and created a new set of plots.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Define a template for all plots
custom_template = {
    "layout": go.Layout(
        font={
            "family": "Lato",
            "size": 16,
            "color": "#1f1f1f",
        },
        title={
            "font": {
                "family": "Lato",
                "size": 32,
                "color": "#1f1f1f",
            },
        },
    )
}

# Text for the attribution
attribution_text = """<b>Data:</b> SMARD/Bundesnetzagentur, 
<b>Graph:</b> Jan Kühn (https://yotka.org), 
<b>License:</b> CC by 4.0"""

current_year = 2023

## Import data from SMARD

In [ ]:
# Files to be imported
# Data can be downloaded at https://www.smard.de/home/downloadcenter/download-marktdaten/
import_files = {
    # Daily electricity production
    "day": "data/Realisierte_Erzeugung_201501010000_202312312359_Tag.csv",
    # Monthly electricity production
    "month": "data/Realisierte_Erzeugung_201501010000_202312312359_Monat.csv",
    # Yearly electricity production
    "year": "data/Realisierte_Erzeugung_201501010000_202312312359_Jahr.csv",
}

# Create empty dictionaries to be filled
data_raw = {}
data = {}

# Loop through the import_files to get the data
for period in import_files:

    # Read CSV
    data_raw[period] = pd.read_csv(
        import_files[period],
        sep=";",
        parse_dates=["Datum"],
        dayfirst=True,
        thousands=".",
        decimal=",",
    )

    # Create empty dictionaries to be filled
    data[period] = {}
    dfs = {}

    # Rename columns
    dfs["abs"] = data_raw[period].rename(
        columns={
            "Datum": "Date",
            "Biomasse [MWh] Berechnete Auflösungen": "Biomass",
            "Wasserkraft [MWh] Berechnete Auflösungen": "Hydro",
            "Wind Offshore [MWh] Berechnete Auflösungen": "Wind Offshore",
            "Wind Onshore [MWh] Berechnete Auflösungen": "Wind Onshore",
            "Photovoltaik [MWh] Berechnete Auflösungen": "Solar",
            "Sonstige Erneuerbare [MWh] Berechnete Auflösungen": "Other renewables",
            "Kernenergie [MWh] Berechnete Auflösungen": "Nuclear",
            "Braunkohle [MWh] Berechnete Auflösungen": "Lignite",
            "Steinkohle [MWh] Berechnete Auflösungen": "Coal",
            "Erdgas [MWh] Berechnete Auflösungen": "Gas",
            "Pumpspeicher [MWh] Berechnete Auflösungen": "Pumped storage",
            "Sonstige Konventionelle [MWh] Berechnete Auflösungen": "Other conventionals",
        }
    )

    # Drop unnecessary columns
    dfs["abs"] = dfs["abs"].drop(columns=["Anfang", "Ende"])

    # Calculate percentages of the whole dataframe
    dfs["pct"] = dfs["abs"].copy().set_index("Date")
    dfs["pct"] = dfs["pct"].div(dfs["pct"].sum(axis=1), axis=0)
    dfs["pct"] = dfs["pct"].reset_index()

    # Loop through dataframes with absolute and relative numbers
    for calc in dfs:

        # Sum data for onshore and offshore wind in one column
        dfs[calc].insert(
            5, "Wind", dfs[calc]["Wind Offshore"] + dfs[calc]["Wind Onshore"]
        )

        # Sum data for coal and lignite in one column
        dfs[calc].insert(10, "Coal & Lignite", dfs[calc]["Coal"] + dfs[calc]["Lignite"])

        # Calculate totals for renewable sources
        dfs[calc].insert(
            15,
            "Renewables",
            dfs[calc]["Wind"]
            + dfs[calc]["Biomass"]
            + dfs[calc]["Hydro"]
            + dfs[calc]["Solar"]
            + dfs[calc]["Other renewables"],
        )

        # Calculate totals for conventional sources
        dfs[calc].insert(
            16,
            "Conventionals",
            dfs[calc]["Nuclear"]
            + dfs[calc]["Coal"]
            + dfs[calc]["Gas"]
            + dfs[calc]["Pumped storage"]
            + dfs[calc]["Other conventionals"],
        )

        # Calculate totals for fossil sources
        dfs[calc].insert(
            17,
            "Fossil",
            dfs[calc]["Coal & Lignite"] + dfs[calc]["Gas"],
        )

        # Add the dataframe to our main data variable
        data[period][calc] = dfs[calc].copy()

## Daily data: Heatmaps
The charts that got my attention in the first place were heatmaps of electricity production representing each day of the year for multiple years. I rebuild the same charts for Germany. It has to be noted, though, that there were no "coal-free" days in Germany in the observed time period. For that reason I opted to rather highlight outliers using quantiles, i.e. both 5% of the lowest and highest daily values are highlighted.

In [ ]:
# Prepare dataframe to plot the heatmaps
df_day_heatmap = data["day"]["pct"].reset_index(drop=True)

# Add columns for Year and Day of Year
df_day_heatmap.insert(1, "Year", df_day_heatmap["Date"].dt.year)
df_day_heatmap.insert(2, "Day", df_day_heatmap["Date"].dt.dayofyear)

# Have a look at the final dataframe
df_day_heatmap.head()

In [ ]:
# Create the heatmap charts using Plotly

# Set width and height
width = 1200
height = 600

# Define some settings for the different types of electricity production
metrics = [
    {
        "name": "Renewables",  # Machine readable name
        "title": "Renewables are getting closer to 60% of electricity production",  # Title
        "add": "(Wind, solar, water, biomass, and others)",  # Additional information for the subtitle
        "color": "#109648",  # Main color
        "color_low": "red",  # Color to highlight low outliers
        "color_high": "#000",  # Color to highlight high outliers
    },
    {
        "name": "Conventionals",
        "title": "Conventional electricity generation declines to about one fourth",
        "add": "(Coal, gas, nuclear, and others)",
        "color": "#333",
        "color_low": "#109648",
        "color_high": "red",
    },
    {
        "name": "Fossil",
        "title": "No end in sight for electricity generation from fossil sources in Germany",
        "add": "(Coal, lignite, and gas)",
        "color": "#333",
        "color_low": "#109648",
        "color_high": "red",
    },
    {
        "name": "Wind",
        "title": (
            f"Wind is at {round(df_day_heatmap[df_day_heatmap['Year'] == current_year]['Wind'].mean() * 100)}% of electricity production "
            f"with daily peaks up to {round(df_day_heatmap[df_day_heatmap['Year'] == current_year]['Wind'].max() * 100)}%"
        ),
        "add": "",
        "color": "#D99AC5",
        "color_low": "red",
        "color_high": "#109648",
    },
    {
        "name": "Hydro",
        "title": "Hydropower does not play a major role in German electricity production",
        "add": "",
        "color": "#124E78",
        "color_low": "red",
        "color_high": "#000",
    },
    {
        "name": "Solar",
        "title": (
            f"Solar power keeps growing with lots of volatility and daily peaks up to "
            f"{round(df_day_heatmap[df_day_heatmap['Year'] == current_year]['Solar'].max() * 100)}%"
        ),
        "add": "",
        "color": "#FCF300",
        "color_low": "red",
        "color_high": "#000",
    },
    {
        "name": "Nuclear",
        "title": "Nuclear energy is on the decline",
        "add": "",
        "color": "#FF9000",
        "color_low": "#109648",
        "color_high": "#000",
    },
    {
        "name": "Coal & Lignite",
        "title": "Coal & lignite declined but were revived since 2021",
        "add": "",
        "color": "#000",
        "color_low": "#109648",
        "color_high": "red",
    },
    {
        "name": "Gas",
        "title": "Gas is on the rise with historic peaks of up to 25% in late 2022",
        "add": "",
        "color": "#D7263D",
        "color_low": "#109648",
        "color_high": "#000",
    },
]

# Start counter to be used in the loop
i = 1

# Loop through the different types
for metric in metrics:

    # Create one subplot for each year
    fig = make_subplots(
        rows=df_day_heatmap["Year"].nunique(),
        cols=1,
        vertical_spacing=0,
    )

    # Set some variables for later use
    row_num = 1  # Row number (used as a counter)
    label_pos = 0.96  # Vertical position of the first year

    # Loop through the years
    for year in df_day_heatmap["Year"].unique():

        # Filter data for current year
        data_year = df_day_heatmap[df_day_heatmap["Year"] == year]

        # Add a trace for current year
        fig.add_trace(
            go.Heatmap(
                y=data_year["Year"],
                x=data_year["Day"],
                z=data_year[metric["name"]],
                coloraxis="coloraxis",  # Use the same coloraxis for all subplots
            ),
            row=row_num,
            col=1,
        )

        # Add years as y-axis labels
        fig.add_annotation(
            xref="paper",
            yref="paper",
            xanchor="left",
            yanchor="top",
            x=-0.04,
            y=label_pos,
            showarrow=False,
            text=str(year),
            borderpad=0,
        )

        # Add yearly average value to the right
        fig.add_annotation(
            xref="paper",
            yref="paper",
            xanchor="left",
            yanchor="top",
            x=1.005,
            y=label_pos,
            showarrow=False,
            text=f"Ø {data_year[metric['name']].mean().round(2):.0%}",
            borderpad=0,
        )

        if year in [2015, current_year]:
            # Add lowest & highest days in 2015 & current year
            df_stats = data_year[metric["name"]]

            # Define thresholds (based on all years)
            thres_low = df_day_heatmap[metric["name"]].quantile(0.05)
            thres_high = df_day_heatmap[metric["name"]].quantile(0.95)

            # Count days below/above thresholds
            count_low = df_stats[df_stats < thres_low].count()
            count_high = df_stats[df_stats > thres_high].count()

            # Set y position depending on the year
            ypos = -0.05 if year == 2015 else -0.08

            # Add annotation
            fig.add_annotation(
                xref="paper",
                yref="paper",
                xanchor="right",
                x=1,
                y=ypos,
                showarrow=False,
                text=f"{year}: <b>{count_low}</b> low days (<{thres_low:.0%}) and <b>{count_high}</b> high days (>{thres_high:.0%})",
                font=dict(size=12),
            )

        # Increase counters
        row_num += 1
        label_pos -= 0.112

    # Add attribution
    fig.add_annotation(
        xref="paper",
        yref="paper",
        x=-0.04,
        y=-0.08,
        showarrow=False,
        text=attribution_text,
        font=dict(size=12),
    )

    # Some final touches
    fig.update_layout(
        title="<b>" + metric["title"] + "</b><br />"
        f"<sup>Daily electricity production share in Germany 2015-{current_year} "
        + metric["add"]
        + "</sup>",
        title_x=0.01,
        title_y=0.94,
        showlegend=False,
        width=width,
        height=height,
        margin=dict(
            autoexpand=True,
            b=40,
            l=55,
            pad=0,
            r=80,
            t=120,
        ),
        template=custom_template,
        coloraxis=dict(  # Define common color axis
            cmin=0,
            cmax=df_day_heatmap[metric["name"]].max(),
            colorscale=[
                [0, metric["color_low"]],
                [  # Calculate lower bound
                    df_day_heatmap[metric["name"]].quantile(0.05)
                    / df_day_heatmap[metric["name"]].max(),
                    "white",
                ],
                [  # Calculate upper bound
                    df_day_heatmap[metric["name"]].quantile(0.95)
                    / df_day_heatmap[metric["name"]].max(),
                    metric["color"],
                ],
                [1, metric["color_high"]],
            ],
            colorbar=dict(
                xanchor="right",
                yanchor="top",
                x=1.01,
                y=1.1,
                orientation="h",
                thickness=10,
                len=0.33,
                tickformat=".0%",
                tickfont=dict(size=12),
            ),
        ),
    )

    # Hide ticks and labels on all axes
    fig.update_xaxes(showticklabels=False, visible=False)
    fig.update_yaxes(showticklabels=False, visible=False)

    # Export chart as PNG and SVG image
    name = metric["name"].replace("&", "").replace("  ", " ").replace(" ", "-").lower()
    fig.write_image(f"export/heatmap-{i:02d}-{name}.png", scale=2)
    fig.write_image(f"export/svg/heatmap-{i:02d}-{name}.svg", scale=2)

    fig.show()

    # Increase counter
    i += 1

## Monthly data
Interestingly, despite many efforts to boost renewable electricity production and fade out conventionals and especially fossil sources, the last two years we have observed a reversal of the trend. Due to different reasons including the Russian invasion of Ukraine and the EU's und Germany's reactions to it, fossil sources for electricity generation have gained importances again. We show that in a chart using monthly data. 

In [ ]:
# Prepare a dataframe to plot it using Plotly
df_plot = data["month"]["pct"].copy()

# Define width and height
width = 1200
height = 600

# Create the Plotly figure
fig = px.area(
    df_plot,
    x="Date",
    y=[
        "Nuclear",
        "Pumped storage",
        "Other conventionals",
        "Gas",
        "Lignite",
        "Coal",
        "Other renewables",
        "Biomass",
        "Hydro",
        "Solar",
        "Wind",
    ],
    width=width,
    height=height,
    template=custom_template,
    color_discrete_map={
        "Nuclear": "#FF9000",
        "Pumped storage": "#ffab2e",
        "Other conventionals": "#ffc74c",
        "Gas": "#c6c6c6",
        "Lignite": "#5e5e5e",
        "Coal": "#000",
        "Other renewables": "#94ffaf",
        "Biomass": "#77ea94",
        "Hydro": "#5acd7a",
        "Solar": "#3bb161",
        "Wind": "#109648",
    },
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.9,
    textangle=-90,
    showarrow=False,
    text="Renewable",
    font=dict(color="#666", size=12),
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.66,
    textangle=-90,
    showarrow=False,
    text="Fossil",
    font=dict(color="#666", size=12),
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.52,
    textangle=-90,
    showarrow=False,
    text="Other",
    font=dict(color="#666", size=12),
)

# Add attribution
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="left",
    yanchor="top",
    x=-0.005,
    y=-0.07,
    showarrow=False,
    text=attribution_text,
    font=dict(size=12),
)

# Some final touches
fig.update_layout(
    title="<b>Still a long way to full renewable electricity in Germany</b><br />"
    "<sup>Share of electricity production 2015-2023 by month</sup>",
    title_x=0.01,
    title_y=0.94,
    showlegend=True,
    margin=dict(b=60, l=60, pad=5, r=25, t=70),
    xaxis=dict(
        title="",
        showgrid=False,
        ticklabelposition="outside right",
        ticks="inside",
    ),
    yaxis=dict(
        title="",
        showgrid=True,
        gridcolor="rgba(0, 0, 0, 0.5)",
        tickformat=".0%",
        tickmode="array",
        tickvals=[0.25, 0.50, 0.75, 1],
    ),
    legend=dict(
        title="",
        traceorder="reversed",
        yanchor="top",
        x=1.04,
        y=0.975,
    ),
)

fig.update_traces(line=dict(width=1))

fig.show()

# Export chart as PNG and SVG image
fig.write_image("export/all-sources-abs.png", scale=2)
fig.write_image("export/svg/all-sources-abs.svg", scale=2)

In [ ]:
# Prepare a dataframe to plot it using Plotly
df_plot = data["month"]["abs"].copy()

# Define series
series = {
    "Conventionals": "#FF9000",
    "Fossil": "#000",
    "Renewables": "#109648",
}
change = {}

# Calculate a rolling average over 3 months
for s in series:
    df_plot[s] = df_plot[s].rolling(window=12).mean()

# Remove rows with NaN values for series.keys()
df_plot = df_plot.dropna(subset=series.keys())

for s in series:
    # Get percentage change from 2015 to 2022
    change[s] = (df_plot[s].iloc[-1] / df_plot[s].iloc[0] - 1) * 100

# Create subplots
fig = make_subplots(
    rows=1,
    cols=3,
    shared_yaxes=True,
    # subplot_titles=[f"<b>{s}</b>" for s in series],
)

# Loop over each subplot
for i, serie in enumerate(series, start=1):
    for s in series:
        if s != serie:
            fig.add_trace(
                go.Scatter(
                    x=df_plot["Date"],
                    y=df_plot[s],
                    name=s,
                    line=dict(color="#eee"),
                ),
                row=1,
                col=i,
            )
    # Add the current line last
    fig.add_trace(
        go.Scatter(
            x=df_plot["Date"],
            y=df_plot[serie],
            name=serie,
            line=dict(color=series[serie]),
        ),
        row=1,
        col=i,
    )

# Generate a list of all years from 2015 to the last year in your data
all_years = (
    pd.date_range(start="2015", end="2023", freq="YS").strftime("%Y-%m-%d").tolist()
)

# Center the labels by setting them to the middle of each year
centered_years = (
    (pd.to_datetime(all_years) + pd.offsets.MonthBegin(6)).strftime("%Y-%m-%d").tolist()
)

fig.update_xaxes(
    showgrid=False,
    tickformat="'%y",
    tick0="2016",
    dtick="M12",
    tickvals=centered_years,
)

# Set y-axis range
fig.update_yaxes(range=[7000000, 23000000])

# Some final touches
fig.update_layout(
    title="<b>Composition of electricity production in Germany changes clearly</b><br />"
    "<sup>Monthly electricity production from <b style='color:#FF9000'>Conventional</b>, "
    "<b style='color:#000'>Fossil</b>, and <b style='color:#109648'>Renewable</b> "
    "sources 2016-2023 (in MWh, rolling average)</sup>",
    title_x=0.01,
    title_y=0.94,
    width=1200,
    height=675,
    margin=dict(b=80, l=60, pad=5, r=25, t=100),
    template=custom_template,
    showlegend=False,
)

# Add attribution
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1,
    y=-0.1,
    showarrow=False,
    text=attribution_text,
    font=dict(size=12),
)

# Add annotation for the last data point of each series, using the change dictionary
for i, s in enumerate(series, start=1):
    yshift = 10 if change[s] > 0 else -10

    fig.add_annotation(
        x=df_plot["Date"].iloc[-1],
        y=df_plot[s].iloc[-1],
        xref=f"x{i}",
        yref=f"y{i}",
        text=f"<b>{'+' if change[s] > 0 else ''}{change[s]:.0f}%</b>",
        showarrow=False,
        yshift=yshift,
        font=dict(color=series[s], size=12),
    )

fig.show()

# Export chart as PNG and SVG image
fig.write_image("export/composition-change.png", scale=2)
fig.write_image("export/svg/composition-change.svg", scale=2)

In [ ]:
# Prepare a dataframe to plot it using Plotly
df_plot = data["month"]["abs"].copy()

series = {
        "Nuclear": "#FF9000",
        "Pumped storage": "#ffab2e",
        "Other conventionals": "#ffc74c",
        "Gas": "#c6c6c6",
        "Lignite": "#5e5e5e",
        "Coal": "#000",
        "Other renewables": "#94ffaf",
        "Biomass": "#77ea94",
        "Hydro": "#5acd7a",
        "Solar": "#3bb161",
        "Wind": "#109648",
    }

# Calculate a rolling average over 3 months
for s in series:
    df_plot[s] = df_plot[s].rolling(window=12).mean()

# Remove rows with NaN values for series.keys()
df_plot = df_plot.dropna(subset=series.keys())

# Create total from keys in series
df_plot["Total"] = df_plot[list(series.keys())].sum(axis=1)

# Calculate percentage change from 2016 to 2023
change = (df_plot["Total"].iloc[-1] / df_plot["Total"].iloc[1] - 1) * 100

# Define width and height
width = 1200
height = 600

# Create the Plotly figure
fig = px.area(
    df_plot,
    x="Date",
    y=list(series.keys()),
    width=width,
    height=height,
    template=custom_template,
    color_discrete_map=series,
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.9,
    textangle=-90,
    showarrow=False,
    text="Renewable",
    font=dict(color="#666", size=12),
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.66,
    textangle=-90,
    showarrow=False,
    text="Fossil",
    font=dict(color="#666", size=12),
)

# Add info to legend
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="right",
    yanchor="top",
    x=1.04,
    y=0.52,
    textangle=-90,
    showarrow=False,
    text="Other",
    font=dict(color="#666", size=12),
)

# Add attribution
fig.add_annotation(
    xref="paper",
    yref="paper",
    xanchor="left",
    yanchor="top",
    x=-0.005,
    y=-0.07,
    showarrow=False,
    text=attribution_text,
    font=dict(size=12),
)

# Annotation for the last data point of Wind
fig.add_annotation(
    x=df_plot["Date"].iloc[-1],
    y=df_plot["Total"].iloc[-1],
    xref="x",
    yref="y",
    text=f"<b>{change:.0f}%</b>",
    showarrow=False,
    xshift=-10,
    yshift=15,
)

# Some final touches
fig.update_layout(
    title="<b>Total German electricity production tends to decline</b><br />"
    "<sup>Monthly electricity production 2016-2023 (in MWh, 12 month rolling average)</sup>",
    title_x=0.01,
    title_y=0.94,
    showlegend=True,
    margin=dict(b=60, l=60, pad=5, r=25, t=70),
    xaxis=dict(
        title="",
        showgrid=True,
        ticklabelposition="outside right",
        ticks="inside",
        range=["2016-01-01", "2023-12-01"],
    ),
    yaxis=dict(
        title="",
        showgrid=False,
    ),
    legend=dict(
        title="",
        traceorder="reversed",
        yanchor="top",
        x=1.04,
        y=0.975,
    ),
)

fig.update_traces(line=dict(width=1))

fig.show()

# Export chart as PNG and SVG image
fig.write_image("export/total-production-abs-rolling.png", scale=2)
fig.write_image("export/svg/total-production-abs-rolling.svg", scale=2)